In [1]:
import pandas as pd
import geopandas as gpd
import scipy.sparse
from fuzzywuzzy import process as fuzz_process
from geopy.distance import great_circle

In [2]:
df_schools = pd.read_csv('df_schools.csv').fillna('')
df_schools['id'] = df_schools.index

In [3]:
#!mkdir -p counties/arcgis
#!cd counties/arcgis && curl --compressed 'https://prod-hub-indexer.s3.amazonaws.com/files/071bc497268b4643b68fcdbde2b13a7e/0/full/4326/071bc497268b4643b68fcdbde2b13a7e_0_full_4326.zip' > data.zip && unzip data.zip

In [4]:
# Note that we make sure to use the same CRS projection that was used in schools dataset
counties = pd.read_csv('counties.csv')
counties.head()

,county
0,Baringo
1,Bomet
2,Siaya
3,Bungoma
4,Kericho


In [5]:
# reverse index for the schools, mapping school (name, lat, long) to the row id in the dataset
schools_map = {(t.name, t.lat, t.long): t.id for t in df_schools.itertuples()}

In [6]:
# data downloaded from ishamba CustomerPlantVillage table. We are only interested in rows where customer has entered school name
data = pd.read_json('data_schools.json')
data.columns = ['id', 'county', 'county_raw', 'school', 'school_raw', 'school_recognized', 'lat', 'long', 'is_complete']
data['school_recognized'] = data['school_recognized'].astype('boolean')
data = data[~data.school_raw.isna()].copy()
data

,id,county,county_raw,school,school_raw,school_recognized,lat,long,is_complete
0,7,NANDI,Nandi,SAMOEI BOYS SECONDARY SCHOOL,Samoei boys,True,0.108394,35.169737,True
1,5,TAITA TAVETA,Taveta,None,Maho secondary,False,NaN,NaN,True
2,4,NAROK,Narok,NAROK HIGH,Narok,True,-1.069555,35.864660,True
3,12,MOMBASA,mombasa,STAR OF THE SEA,star if the sea,True,-4.066130,39.669400,True
4,8,NAKURU,Nakuru,BAHATI PCEA GIRLS,Bahati Girls,True,-0.144056,36.169991,True
...,...,...,...,...,...,...,...,...,...
3592,3620,None,kakamega,KAMASAI,kamasai,True,0.543860,34.879540,True
3593,3619,BUNGOMA,Bungoma,MALINDA SA,Malinda Sa,True,0.769722,34.501334,False
3594,3622,BUNGOMA,Bungoma,None,bridge international academy,False,NaN,NaN,True
3595,3623,None,Nakuru,ELDAMA RAVINE BOARDING PRI,Eldama Ravine day and boarding primary school,True,0.040400,35.722010,True


In [7]:
# where PV service got customer confirmation for county and school, map that back to row id in the schools dataset
data['school_id'] = [schools_map.get((t.school, t.lat, t.long)) for t in data.itertuples()]
data['school_id'] = data['school_id'].astype('Int64')
data

,id,county,county_raw,school,school_raw,school_recognized,lat,long,is_complete,school_id
0,7,NANDI,Nandi,SAMOEI BOYS SECONDARY SCHOOL,Samoei boys,True,0.108394,35.169737,True,23993
1,5,TAITA TAVETA,Taveta,None,Maho secondary,False,NaN,NaN,True,<NA>
2,4,NAROK,Narok,NAROK HIGH,Narok,True,-1.069555,35.864660,True,24095
3,12,MOMBASA,mombasa,STAR OF THE SEA,star if the sea,True,-4.066130,39.669400,True,13722
4,8,NAKURU,Nakuru,BAHATI PCEA GIRLS,Bahati Girls,True,-0.144056,36.169991,True,23830
...,...,...,...,...,...,...,...,...,...,...
3592,3620,None,kakamega,KAMASAI,kamasai,True,0.543860,34.879540,True,16040
3593,3619,BUNGOMA,Bungoma,MALINDA SA,Malinda Sa,True,0.769722,34.501334,False,1635
3594,3622,BUNGOMA,Bungoma,None,bridge international academy,False,NaN,NaN,True,<NA>
3595,3623,None,Nakuru,ELDAMA RAVINE BOARDING PRI,Eldama Ravine day and boarding primary school,True,0.040400,35.722010,True,30429


In [8]:
# before we can score the new matcher, we need to normalize the confirmed county to our new adopted county names standard
data_known = data[~data.school_id.isna()].copy()
data_known2 = data_known[~data_known.county.isna()].copy()
data_known2['county'] = data_known2.county.apply(lambda name: fuzz_process.extractOne(name, counties.county)[0])
data_known2

,id,county,county_raw,school,school_raw,school_recognized,lat,long,is_complete,school_id
0,7,Nandi,Nandi,SAMOEI BOYS SECONDARY SCHOOL,Samoei boys,True,0.108394,35.169737,True,23993
2,4,Narok,Narok,NAROK HIGH,Narok,True,-1.069555,35.864660,True,24095
3,12,Mombasa,mombasa,STAR OF THE SEA,star if the sea,True,-4.066130,39.669400,True,13722
4,8,Nakuru,Nakuru,BAHATI PCEA GIRLS,Bahati Girls,True,-0.144056,36.169991,True,23830
6,10,Nairobi,nairobi,GRANDMAK PRIMARY,bidii primary,True,-1.281340,36.952110,False,32258
...,...,...,...,...,...,...,...,...,...,...
3588,3614,Homa Bay,Homa Bay,WACHARA,Wachara,True,-0.812414,34.306304,True,3409
3589,3616,Homa Bay,Homabay,LORATENG,Loorateng,True,-0.603800,34.529400,True,3052
3590,3621,Uasin Gishu,Uasin Gishu,MATUNDA RC,Matunda rc,True,0.844360,35.136296,True,19524
3593,3619,Bungoma,Bungoma,MALINDA SA,Malinda Sa,True,0.769722,34.501334,False,1635


In [9]:
from typing import Tuple, List, Iterable, Set
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

class Matcher:

    default_remove_regexp = "'"
    default_to_space_regexp = r'[^\w]+'

    def __init__(
        self,
        ngram_range: Tuple[int, int],
        df: pd.DataFrame,
        counties: Iterable[str],
        county_school_matrix: scipy.sparse.spmatrix = None,
        remove_regexp=default_remove_regexp,
        to_space_regexp=default_to_space_regexp,
        stop_words: Iterable[str] = None,
    ):
        self.df = df
        self.remove_regexp = remove_regexp
        self.to_space_regexp = to_space_regexp
        self.stop_words = set(stop_words) if stop_words else {}
        self.vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=ngram_range)
        counties = np.array(counties)
        self.counties_name_to_id = {county_name: i for i, county_name in enumerate(counties)}
        corpus = self._clean(df['name'])
        self.X = self.vectorizer.fit_transform(corpus)
        if county_school_matrix is None:
            county_school_matrix = scipy.sparse.csr_matrix(
                df.county.to_numpy()[np.newaxis, :] == counties[:, np.newaxis])
        self.county_school_matrix = county_school_matrix

    @staticmethod
    def clean(
            s: pd.Series,
            remove_regexp=default_remove_regexp,
            to_space_regexp=default_to_space_regexp,
            stop_words: Set[str] = None
    ) -> pd.Series:
        if remove_regexp:
            s = s.str.replace(remove_regexp, '', regex=True)
        if to_space_regexp:
            s = s.str.replace(to_space_regexp, ' ', regex=True)
        s = s.str.lower().str.strip()
        if stop_words:
            s = s.str.split().apply(
                lambda l: [x for x in l if x not in stop_words]
            ).apply(
                lambda l: ' '.join(l)
            )
        return s

    def _clean(self, s: pd.Series):
        return self.clean(
            s,
            remove_regexp=self.remove_regexp,
            to_space_regexp=self.to_space_regexp,
            stop_words=self.stop_words
        )

    def _match(self, vals: Iterable[str], counties: Iterable[str] = None):
        if not isinstance(vals, pd.Series):
            vals = pd.Series(vals)
        if counties is not None and not isinstance(counties, pd.Series):
            counties = pd.Series(counties)
        Y = self.vectorizer.transform(self._clean(vals))
        ret: scipy.sparse.spmatrix = self.X.dot(Y.transpose())
        if counties is not None:
            county_ids = [self.counties_name_to_id[county_name] for county_name in counties]
            ret = ret.multiply(self.county_school_matrix[county_ids].transpose())
        return ret.toarray()

    def match(
            self,
            vals: Iterable[str],
            n=5,
            counties: Iterable[str] = None
    ) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
        """
        Searches for multiple school names, returning n top matches for each of them.
        The return value is a tuple of three 2x2 ndarrays: ids, scores, and names.
        Each of the arrays has a shape (n, len(vals)), with each column corresponding to
        one of the searched-for vals, and each row to one of the matches returned for that
        val.

        If counties is passed in, it must be an Iterable of the same length as vals, with desired
        county names corresponding to those used to train the model.
        """
        res = self._match(vals, counties)
        ind = res.argsort(axis=0)[:-(n+1):-1, :]
        return ind, np.take_along_axis(res, ind, axis=0), self.df['name'].to_numpy()[ind]

    def match_df(
            self,
            val: str,
            n=5,
            distance_from: Tuple[int, int] = None,
            county: str = None
    ) -> pd.DataFrame:
        """
        Searches for a single school name and returns nice dataframe with top n matches.
        The distance shown is distance in km from the top match, or from the `distance_from`
        geopoint provided as (lat, long) tuple.
        """
        res = self._match([val], [county] if county is not None else None)
        res = res[:, 0]
        ind: np.ndarray = res.argsort()[-n:]
        ind = ind[::-1]
        df: pd.DataFrame = self.df.iloc[ind].copy()
        best_lat, best_long = distance_from if distance_from else df.iloc[0][['lat', 'long']]
        df['dist'] = [great_circle((t.lat, t.long), (best_lat, best_long)).km for t in df.itertuples()]
        df['score'] = res[ind]
        return df


In [10]:
# figure out stop words
from collections import Counter
import itertools
names = Matcher.clean(df_schools.name)
c = Counter(itertools.chain.from_iterable([s.split() for s in names]))
# count as stop word anything that shows up more than 100 times
stop_words, _ = zip(*itertools.takewhile(lambda t: t[1] > 100, c.most_common()))
stop_words = set(stop_words)
# add few more things which are not that common, but still not useful to match on
stop_words.update(['and', 'schools'])


In [11]:
county_school_matrix = scipy.sparse.load_npz('county_school_matrix.npz')
county_school_matrix

<47x37742 sparse matrix of type '<class 'numpy.float64'>'
	with 62531 stored elements in Compressed Sparse Row format>

In [12]:
m23sw = Matcher(ngram_range=(2,3), df=df_schools, counties=counties.county, stop_words=stop_words, county_school_matrix=county_school_matrix)
m234sw = Matcher(ngram_range=(2,4), df=df_schools, counties=counties.county, stop_words=stop_words, county_school_matrix=county_school_matrix)
m3sw = Matcher(ngram_range=(3,3), df=df_schools, counties=counties.county, stop_words=stop_words, county_school_matrix=county_school_matrix)
m23 = Matcher(ngram_range=(2,3), df=df_schools, counties=counties.county, stop_words=None, county_school_matrix=county_school_matrix)

In [13]:
def score_matcher2(m: Matcher, data: pd.DataFrame, n=5) -> pd.Series:
    ids, scores, _ = m.match(data.school_raw, n, counties=data.county)
    ranks = pd.DataFrame(scores).rank(ascending=False)

    ret_scores = [2*n] * len(data)  # default score when desired match was not found
    desired_ids = data.school_id.to_numpy()
    # we subtract the desired id from all match ids; each 0 corresponds to finding the desired match
    # then we find those zeros, note their indexes, and fill out the rank of the match
    for row_ind, col_ind in np.argwhere((ids - desired_ids[np.newaxis, :]) == 0):
        ret_scores[col_ind] = ranks.iloc[row_ind, col_ind]

    return pd.Series(ret_scores)


In [14]:
scores_m23sw = pd.Series(score_matcher2(m23sw, data_known2))
scores_m234sw = pd.Series(score_matcher2(m234sw, data_known2))
scores_m3sw = pd.Series(score_matcher2(m3sw, data_known2))
scores_m23 = pd.Series(score_matcher2(m23, data_known2))

In [15]:
scores_df = pd.DataFrame({'m23sw': scores_m23sw, 'm234sw': scores_m234sw, 'm3sw': scores_m3sw, 'm23': scores_m23})
scores_df.describe(percentiles=[.85, .875, .9, .925, .95])

,m23sw,m234sw,m3sw,m23
count,1805.000000,1805.000000,1805.000000,1805.000000
mean,2.371745,2.379778,2.358726,1.869252
std,2.728378,2.739009,2.701007,2.132611
min,1.000000,1.000000,1.000000,1.000000
50%,1.500000,1.500000,1.500000,1.000000
85%,3.000000,3.000000,3.000000,2.000000
87.5%,3.000000,3.000000,3.000000,2.500000
90%,10.000000,10.000000,10.000000,3.000000
92.5%,10.000000,10.000000,10.000000,4.000000
95%,10.000000,10.000000,10.000000,10.000000


In [16]:
m23sw.match_df('cheglet', county='Baringo', n=5)

,name,County,SUB_COUNTY,Ward,long,lat,county,id,dist,score
2204,CHEGILET,Elgeyo Marakwet,Keiyo North,Emsoo,35.596590,0.830610,Elgeyo Marakwet,2204,0.000000,0.491951
25510,CHEGILET SECINDARY SCHOOL,Elgeyo Marakwet,Keiyo North,Emsoo,35.601344,0.827696,Elgeyo Marakwet,25510,0.619978,0.333585
475,LEGETETWET,Baringo,Mogotio,Mogotio,35.897977,0.037486,Baringo,475,94.343845,0.210363
146,CHEMURA,Baringo,Baringo North,Barwessa,35.624080,0.586429,Baringo,146,27.323224,0.189765
22,KAPCHEREBET,Baringo,Baringo Central,Kapropita,35.719200,0.452350,Baringo,22,44.214820,0.181090
